In [1]:
import os
import numpy as np

#from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm

import operator

In [2]:
# Load Glove Embedding
gloveFile = "glove.6B/glove.6B.300d.txt"
glove_model = {}

with open(gloveFile, 'r') as f:
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        glove_model[word] = embedding
    print "Done.",len(glove_model)," words loaded!"  

print glove_model['unk']

Done. 400000  words loaded!
[  3.00710000e-01  -4.68670000e-01  -2.06170000e-01  -8.09780000e-01
  -2.38890000e-01   2.43290000e-01   1.65380000e-02  -3.56870000e-02
  -2.23060000e-01   9.51890000e-01  -3.22730000e-01   2.19800000e-01
  -6.75240000e-02  -3.72200000e-01  -3.97180000e-01  -4.38610000e-01
   1.19670000e-01  -2.99640000e-01   2.84370000e-02  -8.75440000e-02
   1.65690000e-01  -4.94510000e-01  -6.20110000e-01  -1.65740000e-01
  -9.72180000e-02  -9.94740000e-02  -8.03070000e-02  -3.93380000e-01
  -2.41950000e-01   3.20230000e-01  -5.33200000e-01  -4.01840000e-01
  -6.71350000e-01  -7.85610000e-02   5.55460000e-01   2.99970000e-01
  -9.96500000e-02  -6.70350000e-01   1.26690000e-01  -1.86180000e-01
  -6.26210000e-02   4.52900000e-01   3.92650000e-01   2.41210000e-01
  -4.14740000e-01  -6.18900000e-01  -1.04120000e-01  -3.10430000e-01
  -6.67880000e-03  -8.32480000e-01   6.51500000e-01   9.01810000e-01
   2.41460000e-02  -7.07660000e-02  -3.95800000e-01  -3.64870000e-01
  -2.3

In [3]:
print glove_model['organization']

[ -4.57080000e-02  -4.08090000e-03  -1.71850000e-01   1.57380000e-01
  -1.54510000e-01   1.29430000e-01  -3.71710000e-01   2.87720000e-01
  -1.77560000e-01  -1.74180000e+00   6.77970000e-01   1.95320000e-01
  -2.10470000e-01   3.12780000e-02  -2.72330000e-01  -9.39930000e-02
   2.69210000e-01   3.84920000e-01  -5.40090000e-01  -1.63370000e-01
   3.35600000e-01   4.30180000e-01   5.00560000e-02   1.74400000e-02
  -5.07370000e-02   1.41920000e-01   1.30980000e-01   1.13790000e-01
  -3.73490000e-01   2.58930000e-01   3.53490000e-01   1.49580000e-01
  -5.85640000e-03  -2.07550000e-01  -1.93410000e-01   4.63010000e-02
   3.92770000e-01  -2.71260000e-02  -3.01780000e-01  -5.89790000e-01
  -2.54810000e-02   1.23350000e-01   4.40330000e-01   3.42790000e-01
  -1.65740000e-01  -2.59720000e-01  -3.33980000e-01   4.92420000e-02
  -6.27630000e-02  -3.90970000e-01  -7.25190000e-02   2.46290000e-01
   5.15210000e-02   4.74460000e-02  -3.80240000e-01  -1.71270000e-01
  -3.02750000e-01   7.29570000e-01

In [6]:
# Prepare dataset for NLTK HMM Tagger
# Divide in train and test files [80:20] 

# Directory having content
doc_dir = 'Data/content'

train_file_list = []
test_file_list = []

for f in os.listdir(doc_dir):
    #Random Sampling
    if np.random.uniform(0,1)< 0.8:
        train_file_list.append(f)
    else:
        test_file_list.append(f)

print len(train_file_list)
print len(test_file_list)

1066
240


In [7]:
# Directory having tags
tag_dir = 'Data/new_tags'

tagEmbedding = dict()
tagCount = dict()
win_size = 2
word_dim = 300
unknown_words = set()

for f in train_file_list:
    #print 'Processing file '+f
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            length = min(len(word_in_file), len(tag_in_file))
            for j in xrange(length):
                # Get the word embedding of the word
                word_embed = np.zeros(word_dim)
                if (j>(win_size-1)) and (j<length-win_size):
                    replace_unk = 'unk'
                    for k in xrange(2*win_size + 1):
                        if word_in_file[j-win_size+k].lower() in glove_model:
                            word_embed = np.concatenate((word_embed, glove_model[word_in_file[j-win_size+k].lower()]), axis=0)
                        else:
                            unknown_words.add(word_in_file[j-win_size+k])
                            #word_embed = np.concatenate((word_embed, glove_model['unk']), axis = 0)  
                            word_embed = np.concatenate((word_embed, glove_model[replace_unk]), axis = 0)  
                    # Create embdedding of Tag    
                    if tag_in_file[j] in tagEmbedding:
                        tagEmbedding[tag_in_file[j]] = np.add(tagEmbedding[tag_in_file[j]], word_embed)
                        tagCount[tag_in_file[j]] = tagCount[tag_in_file[j]] + 1
                    else:
                        tagEmbedding[tag_in_file[j]] = word_embed
                        tagCount[tag_in_file[j]] = 1
                    

In [8]:
for key in tagCount:
    tagEmbedding[key] = tagEmbedding[key]/tagCount[key]                   

In [9]:
print tagCount

print '\nPER_Assoc\n'
print tagEmbedding['PER_Assoc']
print '\nORG_Assoc\n'
print tagEmbedding['ORG_Assoc']
print '\n'
print tagEmbedding['PER_Others']
print '\n'
print tagEmbedding['Victim']
print '\n'
print tagEmbedding['LOC_Event']
print '\n'
print tagEmbedding['O']
print '\n'
print tagEmbedding['ORG_Accused']
print '\n'
print tagEmbedding['PER_Victim']
print '\n'
print tagEmbedding['ORG_Victim']
print '\n'
print tagEmbedding['LOC_Others']
print '\n'
print tagEmbedding['ORG_Others']
print '\n'
print tagEmbedding['PER_Accused']
print '\n'
print tagEmbedding['LOC_Accused']
print '\n'
print tagEmbedding['LOC_Assoc']
print '\n'
print tagEmbedding['LOC_Victim']
print '\n'
print tagEmbedding['Event']
print '\n'
print tagEmbedding['Location']

{'PER_Assoc': 168, 'ORG_Assoc': 51, 'PER_Others': 7114, 'Victim': 27, 'LOC_Event': 5346, 'Accused': 192, 'O': 351447, 'ORG_Accused': 2427, 'PER_Victim': 1066, 'ORG_Victim': 321, 'LOC_Others': 7178, 'ORG_Others': 9286, 'PER_Accused': 3627, 'LOC_Accused': 687, 'LOC_Assoc': 314, 'LOC_Victim': 194, 'Event': 313, 'Location': 544}

PER_Assoc

[ 0.          0.          0.         ..., -0.09482873 -0.20728944
  0.03050485]

ORG_Assoc

[ 0.          0.          0.         ..., -0.10847407 -0.18233476
 -0.01068716]


[ 0.          0.          0.         ..., -0.12191061 -0.26938584
  0.14030579]


[ 0.          0.          0.         ..., -0.22678321 -0.17522766
  0.00934241]


[ 0.          0.          0.         ..., -0.16698668 -0.17334857
  0.08105636]


[ 0.          0.          0.         ..., -0.16366825 -0.18297412
  0.03669295]


[ 0.          0.          0.         ..., -0.09672384 -0.1517116
 -0.09163813]


[ 0.          0.          0.         ..., -0.1731721  -0.18890204
  0.07259167

In [22]:
# Test

# Create dictionary of words

all_word_dict = dict()
all_word_set = set()

for f in test_file_list:
    _word_file_path = os.path.join(doc_dir, f)
    _lines_in_word_file = []
    with open(_word_file_path, "rt") as _word_file:        
        for _line in _word_file:
            _lines_in_word_file.append(_line)
        _num_lines = len(_lines_in_word_file)
        for i in xrange(_num_lines):
            _word_in_file = _lines_in_word_file[i].split()
            _words_len = len(_word_in_file)
            for j in xrange(_words_len):
                all_word_set.add(_word_in_file[j])

_word_list = list(all_word_set)
for i in xrange(len(_word_list)):
    all_word_dict[_word_list[i]] = i

topN = 5
Precision = dict()
Recall = dict()
num_docs_dont_have_tag = dict()

Precision['PER_Others'] = 0
Recall['PER_Others'] = 0
Precision['PER_Victim'] = 0
Recall['PER_Victim'] = 0
Precision['PER_Accused'] = 0
Recall['PER_Accused'] = 0
Precision['ORG_Victim'] = 0
Recall['ORG_Victim'] = 0
Precision['ORG_Accused'] = 0
Recall['ORG_Accused'] = 0
Precision['ORG_Others'] = 0
Recall['ORG_Others'] = 0
Precision['LOC_Accused'] = 0
Recall['LOC_Accused'] = 0
Precision['LOC_Others'] = 0
Recall['LOC_Others'] = 0
Precision['LOC_Event'] = 0
Recall['LOC_Event'] = 0
Precision['LOC_Victim'] = 0
Recall['LOC_Victim'] = 0
Precision['PER_Assoc'] = 0
Recall['PER_Assoc'] = 0
Precision['ORG_Assoc'] = 0
Recall['ORG_Assoc'] = 0
Precision['Location'] = 0
Recall['Location'] = 0
Precision['Event'] = 0
Recall['Event'] = 0
Precision['LOC_Assoc'] = 0
Recall['LOC_Assoc'] = 0
Precision['O'] = 0
Recall['O'] = 0
Precision['Victim'] = 0
Recall['Victim'] = 0
Precision['Accused'] = 0
Recall['Accused'] = 0

num_docs_dont_have_tag['PER_Others'] = 0
num_docs_dont_have_tag['PER_Victim'] = 0 
num_docs_dont_have_tag['PER_Accused'] = 0    
num_docs_dont_have_tag['ORG_Victim'] = 0
num_docs_dont_have_tag['ORG_Accused'] = 0
num_docs_dont_have_tag['ORG_Others'] = 0
num_docs_dont_have_tag['LOC_Accused'] = 0
num_docs_dont_have_tag['LOC_Others'] = 0
num_docs_dont_have_tag['LOC_Event'] = 0
num_docs_dont_have_tag['LOC_Victim'] = 0
num_docs_dont_have_tag['PER_Assoc'] = 0
num_docs_dont_have_tag['ORG_Assoc'] = 0
num_docs_dont_have_tag['LOC_Assoc'] = 0
num_docs_dont_have_tag['Event'] = 0
num_docs_dont_have_tag['O'] = 0
num_docs_dont_have_tag['Victim'] = 0
num_docs_dont_have_tag['Location'] = 0
num_docs_dont_have_tag['Accused'] = 0


for f in test_file_list:
    print 'Processing file '+f
    relevant = dict()
    retrieved = dict()
    sorted_retrieved = dict()
    retrieved_count = dict()
    relevant['PER_Others'] = set()
    retrieved['PER_Others'] = dict()
    relevant['PER_Victim'] = set()
    retrieved['PER_Victim'] = dict()
    relevant['PER_Accused'] = set()
    retrieved['PER_Accused'] = dict()
    relevant['ORG_Victim'] = set()
    retrieved['ORG_Victim'] = dict()
    relevant['ORG_Accused'] = set()
    retrieved['ORG_Accused'] = dict()
    relevant['ORG_Others'] = set()
    retrieved['ORG_Others'] = dict()
    relevant['LOC_Accused'] = set()
    retrieved['LOC_Accused'] = dict()
    relevant['LOC_Others'] = set()
    retrieved['LOC_Others'] = dict()
    relevant['LOC_Event'] = set()
    retrieved['LOC_Event'] = dict()
    relevant['LOC_Victim'] = set()
    retrieved['LOC_Victim'] = dict()    
    relevant['PER_Assoc'] = set()
    retrieved['PER_Assoc'] = dict()
    relevant['ORG_Assoc'] = set()
    retrieved['ORG_Assoc'] = dict()
    relevant['LOC_Assoc'] = set()
    retrieved['LOC_Assoc'] = dict()
    relevant['Event'] = set()
    retrieved['Event'] = dict()
    relevant['Victim'] = set()
    retrieved['Victim'] = dict()
    relevant['Location'] = set()
    retrieved['Location'] = dict()
    relevant['O'] = set()
    retrieved['O'] = dict()
    relevant['Accused'] = set()
    retrieved['Accused'] = dict()
    
    
    sorted_retrieved['PER_Others'] = list()    
    sorted_retrieved['PER_Victim'] = list()    
    sorted_retrieved['PER_Accused'] = list()    
    sorted_retrieved['ORG_Victim'] = list()    
    sorted_retrieved['ORG_Accused'] = list()    
    sorted_retrieved['ORG_Others'] = list()    
    sorted_retrieved['LOC_Accused'] = list()    
    sorted_retrieved['LOC_Others'] = list()    
    sorted_retrieved['LOC_Event'] = list()
    sorted_retrieved['LOC_Victim'] = list()    
    sorted_retrieved['PER_Assoc'] = list()    
    sorted_retrieved['ORG_Assoc'] = list()    
    sorted_retrieved['LOC_Assoc'] = list()
    sorted_retrieved['Event'] = list()
    sorted_retrieved['O'] = list()
    sorted_retrieved['Victim'] = list()
    sorted_retrieved['Location'] = list()
    sorted_retrieved['Accused'] = list()
    
    retrieved_count['PER_Others'] = dict()    
    retrieved_count['PER_Victim'] = dict()    
    retrieved_count['PER_Accused'] = dict()    
    retrieved_count['ORG_Victim'] = dict()    
    retrieved_count['ORG_Accused'] = dict()    
    retrieved_count['ORG_Others'] = dict()    
    retrieved_count['LOC_Accused'] = dict()    
    retrieved_count['LOC_Others'] = dict()    
    retrieved_count['LOC_Event'] = dict()
    retrieved_count['LOC_Victim'] = dict()    
    retrieved_count['PER_Assoc'] = dict()    
    retrieved_count['ORG_Assoc'] = dict()    
    retrieved_count['LOC_Assoc'] = dict()
    retrieved_count['Event'] = dict()
    retrieved_count['O'] = dict()
    retrieved_count['Victim'] = dict()
    retrieved_count['Location'] = dict()
    retrieved_count['Accused'] = dict()

    
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            
            # This is number of all words which is N 
            length = min(len(word_in_file), len(tag_in_file))                
            for j in xrange(length):
                # Get the word embedding of the word
                word_embed = np.zeros(word_dim)
                
                if (j>(win_size-1)) and (j<length-win_size):
                    replace_unk = 'unk'
                    for k in xrange(2*win_size + 1):
                        if word_in_file[j-win_size+k].lower() in glove_model:
                            word_embed = np.concatenate((word_embed, glove_model[word_in_file[j-win_size+k].lower()]), axis = 0)
                        else:
                            word_embed = np.concatenate((word_embed, glove_model['unk']), axis=0)  
                    
                    for tag in tagEmbedding:
                        dist = 1 - (dot(tagEmbedding[tag], word_embed)/(norm(tagEmbedding[tag]) * norm(word_embed)))
                        if word_in_file[j] in retrieved[tag].keys():
                            retrieved[tag][word_in_file[j]] = retrieved[tag][word_in_file[j]]+dist
                            retrieved_count[tag][word_in_file[j]] = retrieved_count[tag][word_in_file[j]] + 1
                        else:
                            retrieved[tag][word_in_file[j]] = dist
                            retrieved_count[tag][word_in_file[j]] = 1
                        
                    relevant[tag_in_file[j]] =  word_in_file[j]
   
    #take the average of score of tags (TotalScore/Occurences)
    for tag in tagEmbedding:
        for key in retrieved[tag]:
            retrieved[tag][key] = retrieved[tag][key]/retrieved_count[tag][key]  
    
    # Sort the retrieved results by score
    for tag in tagEmbedding:
        sorted_retrieved[tag] = sorted(retrieved[tag].items(), key=operator.itemgetter(1), reverse=True)
    
    #Calculate Precison and Recall for this file
    try:
        for tag in tagEmbedding:
            tp = 0
            if len(relevant[tag]) == 0:
                num_docs_dont_have_tag[tag] = num_docs_dont_have_tag[tag] +1
            else:
                for i in xrange(topN):
                    if sorted_retrieved[tag][i][0] in relevant[tag]:
                        tp = tp+1
                Precision[tag] = Precision[tag] + (float(tp*100)/topN)
                Recall[tag] = Recall[tag]+ (float(tp*100)/len(relevant[tag]))
                
    except IndexError:
        print f
        
        

Processing file 2012_6_11_st-43.txt
Processing file ev_051_st_007.txt
Processing file ev_029_st_001.txt
Processing file 2012_8_2_st-724.txt
Processing file ev_031_st_001.txt
Processing file ev_087_st_016.txt
Processing file ev_031_st_003.txt
Processing file 2011_9_14_st-2.txt
Processing file 2011_7_23_st-1.txt
Processing file ev_032_st_002.txt
Processing file ev_094_st_004.txt
Processing file 2012_8_6_st-1.txt
Processing file 2013_5_8_st-59.txt
Processing file ev_092_st_015.txt
Processing file 2011_3_10_st-144.txt
Processing file 2010_9_4_st-331.txt
Processing file ev_075_st_002.txt
Processing file 2011_8_16_st-674.txt
Processing file ev_074_st_001.txt
Processing file 2011_9_23_st-45.txt
Processing file 2011_7_17_st-56.txt
Processing file 2012_5_3_st-53.txt
Processing file 2013_3_2_st-40.txt
Processing file ev_006_st_001.txt
Processing file ev_042_st_006.txt
Processing file ev_081_st_011.txt
Processing file ev_093_st_026.txt
Processing file ev_075_st_009.txt
Processing file 2012_9_6_st

Processing file ev_075_st_024.txt
Processing file ev_089_st_003.txt
Processing file 2014_11_12_st-4002.txt


In [23]:
num_test_file = len(test_file_list)
for tag in tagEmbedding:
    try:
        precision = float(Precision[tag])/(num_test_file-num_docs_dont_have_tag[tag])
        recall = float(Recall[tag])/(num_test_file-num_docs_dont_have_tag[tag])
        f1 = float(2*precision*recall)/(precision+recall)
        print tag, str(precision)+'%', str(recall)+'%', f1
    except:
        print tag

PER_Assoc 1.33333333333% 0.833333333333% 1.02564102564
ORG_Assoc
PER_Others 0.110497237569% 0.110497237569% 0.110497237569
Victim
LOC_Event 1.3% 0.886849700008% 1.0543976662
Accused 1.33333333333% 0.952380952381% 1.11111111111
O 0.338983050847% 0.548157115954% 0.418910062837
ORG_Accused 0.582524271845% 0.380258899676% 0.460145223138
PER_Victim 1.05263157895% 0.921052631579% 0.982456140351
ORG_Victim 1.25% 0.892857142857% 1.04166666667
LOC_Others 0.820512820513% 0.781237281237% 0.800393525079
ORG_Others 0.398009950249% 0.394456289979% 0.396225152266
PER_Accused 1.31578947368% 1.07038429407% 1.18046758033
LOC_Accused 1.11111111111% 0.705467372134% 0.862998921251
LOC_Assoc 2.30769230769% 2.69230769231% 2.48520710059
LOC_Victim 3.63636363636% 3.57142857143% 3.6036036036
Event
Location 0.625% 0.520833333333% 0.568181818182
